In [1]:
import tensorflow as tf
import numpy as np
import urllib
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-09-12 13:12:17.392088: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-12 13:12:17.392108: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

sentences = []
labels = []
# YOUR CODE HERE
with open('sarcasm.json', 'r') as f:
    data = json.load(f)

for row in data:
    sentences.append(row['headline'])
    labels.append(row['is_sarcastic'])

In [3]:
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

sentence_stop = []
for sentence in sentences:
    sentence_stop.append(' '.join([word for word in sentence.split() if word not in stopwords]))

In [4]:
# Tokenize and pad

train_sentences = sentences[:training_size]
train_labels = labels[:training_size]
train_labels = np.array(train_labels)

val_sentences = sentences[training_size:]
val_labels = labels[training_size:]
val_labels = np.array(val_labels)

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

training_sequences = tokenizer.texts_to_sequences(train_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [8]:
# Params
dense_dim = 8

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.7),
    # tf.keras.layers.Conv1D(filters, kernel_size, activation='relu'),
    # tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    # tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

es = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model.fit(training_padded,
          train_labels,
          epochs=20,
          validation_data=(val_padded, val_labels),
          batch_size=64,
          callbacks=[es])

Epoch 1/20
313/313 [==============================] - 21s 58ms/step - loss: 0.5251 - accuracy: 0.7168 - val_loss: 0.3998 - val_accuracy: 0.8128
Epoch 2/20
313/313 [==============================] - 19s 59ms/step - loss: 0.4082 - accuracy: 0.8180 - val_loss: 0.4044 - val_accuracy: 0.8143
Epoch 3/20
313/313 [==============================] - 19s 61ms/step - loss: 0.3718 - accuracy: 0.8370 - val_loss: 0.3904 - val_accuracy: 0.8201
Epoch 4/20
313/313 [==============================] - 20s 65ms/step - loss: 0.3520 - accuracy: 0.8454 - val_loss: 0.3854 - val_accuracy: 0.8267
Epoch 5/20
313/313 [==============================] - 24s 77ms/step - loss: 0.3372 - accuracy: 0.8553 - val_loss: 0.3819 - val_accuracy: 0.8278
Epoch 6/20
313/313 [==============================] - 24s 77ms/step - loss: 0.3238 - accuracy: 0.8614 - val_loss: 0.3859 - val_accuracy: 0.8269
Epoch 7/20
313/313 [==============================] - 24s 77ms/step - loss: 0.3136 - accuracy: 0.8654 - val_loss: 0.3827 - val_accuracy: